In [35]:
from __future__ import print_function
import cv2 as cv
import os
import numpy as np

# Step 1: Read images from folder and apply background subtraction
def process_images_with_background_subtraction(image_folder, output_folder, algo='MOG2', background_image_path=None, num_passes=3):
    # Select the background subtraction algorithm with custom sensitivity settings
    if algo == 'MOG2':
        backSub = cv.createBackgroundSubtractorMOG2(history=500)  # Lower varThreshold for sensitivity
    elif algo=="GMG":
        backSub = cv.bgsegm.createBackgroundSubtractorGMG(); 
    else:
         backSub = cv.createBackgroundSubtractorKNN(history=1000, dist2Threshold=600.0, detectShadows=False) 

    # Load the background image if provided
    if background_image_path and os.path.exists(background_image_path):
        print(f"Loading existing background image from: {background_image_path}")
        existing_bg = cv.imread(background_image_path)
        if existing_bg is not None:
            existing_bg = cv.resize(existing_bg, (640, 640))  # Resize to the expected size
            backSub.apply(existing_bg, learningRate=0)  # Initialize with existing background

    # Get all image file names from the folder and sort them by their numeric value
    images = [img for img in os.listdir(image_folder) if img.endswith((".png", ".jpg", ".jpeg"))]
    
    # Sort images based on numeric part of the filename (e.g., 0, 1, 2, etc.)
    images.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))  # Sort by number in file name

    # Check if there are any images in the folder
    if len(images) == 0:
        print("No images found in the folder.")
        return

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process the images for the number of passes
    for pass_num in range(num_passes):
        print(f"\nStarting pass {pass_num + 1}/{num_passes}\n")
        
        # Loop through each image in the folder
        for idx, image in enumerate(images):
            image_path = os.path.join(image_folder, image)
            
            # Print the file name being read
            # print(f"Reading file: {image}")
            
            frame = cv.imread(image_path)

            if frame is None:
                print(f"Error reading image {image_path}. Skipping.")
                continue

            # Resize the image to 640x640
            frame_resized = cv.resize(frame, (640, 640))

            # Apply background subtraction with a low learning rate to prevent quick adaptation
            fgMask = backSub.apply(frame_resized, learningRate=0.00)  # Low learning rate to avoid including slow-moving cars

            # Display the foreground mask
            cv.imshow('Foreground Mask', fgMask)

            # Every 50 frames, display and save the background model
            if (idx + 1) % 50 == 0:  # Show and save background every 50 frames
                bgImage = backSub.getBackgroundImage()
                if bgImage is not None:
                    # Display the background model
                    cv.imshow('Background Model', bgImage)
                    
                    # Save the background image
                    background_save_path = os.path.join(output_folder, f'background_image_pass_{pass_num + 1}.jpg')
                    cv.imwrite(background_save_path, bgImage)
                    # print(f"Saved background image at: {background_save_path}")

            # Exit if 'q' is pressed
            keyboard = cv.waitKey(30)  # Adjust delay as necessary
            if keyboard == ord('q') or keyboard == 27:
                break

    cv.destroyAllWindows()

    # Save the final background model
    bgImage = backSub.getBackgroundImage()
    final_save_path = os.path.join(output_folder, "final_background.jpg")
    if bgImage is not None:
        cv.imwrite(final_save_path, bgImage)
        # print(f"Final background image saved at: {final_save_path}")

# Example usage: Process images from a folder with background subtraction and save the background images
image_folder = r'C:\Users\Welcome\Documents\Kouby\M.S.Robo- Georgia Tech\GATECH LABS\SHREYAS_LAB\SSTA_2024_25\Pneuma_dataset\pneuma_20181029_0_1_D2_0900_0930_raw\train\camera_0'
output_folder = r'C:\Users\Welcome\Documents\Kouby\M.S.Robo- Georgia Tech\GATECH LABS\SHREYAS_LAB\SSTA_2024_25\Pneuma_dataset\pneuma_20181029_0_1_D2_0900_0930_raw\train'  # Folder where background images will be saved
background_image_path = r'C:\Users\Welcome\Documents\Kouby\M.S.Robo- Georgia Tech\GATECH LABS\SHREYAS_LAB\SSTA_2024_25\Pneuma_dataset\pneuma_20181029_0_1_D2_0900_0930_raw\train\background_image.jpg'  # Path to a previously saved background image

# Process the images with MOG2 background subtraction and save background images, optionally loading a background
process_images_with_background_subtraction(image_folder, output_folder, algo='KNN', background_image_path=None, num_passes=1)  # Change to 'KNN' if needed



Starting pass 1/1



In [4]:
# import cv2
# import os

# # Path to the directory with images
# folder_path = r'Pneuma_dataset\pneuma_main_20181029_0_1_D2_0900_0930\train\_MOG_t2no_15\camera_0'

# # Get list of image files in the folder starting with "t2no"
# image_files = [f for f in os.listdir(folder_path) if f.startswith('t2no') and f.endswith(('png', 'jpg', 'jpeg'))]
# image_files.sort()  # Ensure the images are displayed in order

# # Multiply each image by 15 and display as a video
# for img_file in image_files:
#     img_path = os.path.join(folder_path, img_file)
#     img = cv2.imread(img_path)
    
#     # Ensure the image is read properly and multiply the pixel values
#     if img is not None:
#         # Clip values to stay in valid range [0, 255]
#         img_multiplied = cv2.convertScaleAbs(img, alpha=10)  # Adjust multiplier
        
#         # Display the image using OpenCV
#         cv2.imshow('Video Simulation', img_multiplied)
        
#         # Wait for 50ms between frames to simulate video playback
#         if cv2.waitKey(50) & 0xFF == ord('q'):
#             break  # Press 'q' to exit the video early

# # Close all OpenCV windows
# cv2.destroyAllWindows()


In [7]:
import cv2
import os
import numpy as np

# Paths to the directories with images
folder_path_t2no = r'Pneuma_dataset\pneuma_main_20181029_0_1_D2_0900_0930\train\_MOG_t2no_15\camera_1'
folder_path_image = r'Pneuma_dataset\pneuma_main_20181029_0_1_D2_0900_0930\train\camera_1'

# Path to save the video
output_folder = r'Pneuma_dataset\t2no_pneuma_results'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Video output file path
video_output_path = os.path.join(output_folder, 'concatenated_pneuma_results.avi')

# Get list of image files in both folders
image_files_t2no = [f for f in os.listdir(folder_path_t2no) if f.endswith(('png', 'jpg', 'jpeg'))]
image_files_image = [f for f in os.listdir(folder_path_image) if f.endswith(('png', 'jpg', 'jpeg'))]

# Ensure the images are sorted and that both folders have matching image counts
image_files_t2no.sort()
image_files_image.sort()

# Check if both folders have the same number of images
if len(image_files_t2no) != len(image_files_image):
    raise ValueError("The number of images in the two folders does not match!")

# Check if there are images to process
if len(image_files_t2no) > 0:
    # Get the frame size from the first image
    first_image_t2no_path = os.path.join(folder_path_t2no, image_files_t2no[0])
    first_image_t2no = cv2.imread(first_image_t2no_path)
    height, width, layers = first_image_t2no.shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for .avi file
    video_writer = cv2.VideoWriter(video_output_path, fourcc, 20.0, (width * 2, height))  # Concatenated width (double)

    # Process each pair of images and concatenate them
    for img_file_t2no, img_file_image in zip(image_files_t2no, image_files_image):
        img_t2no_path = os.path.join(folder_path_t2no, img_file_t2no)
        img_image_path = os.path.join(folder_path_image, img_file_image)

        # Read images from both folders
        img_t2no = cv2.imread(img_t2no_path)
        img_image = cv2.imread(img_image_path)

        # Ensure both images are read properly
        if img_t2no is not None and img_image is not None:
            # Multiply the t2no image pixel values by 20 and clip to valid range [0, 255]
            img_t2no_multiplied = cv2.convertScaleAbs(img_t2no, alpha=10)  # Adjust multiplier as necessary

            # Concatenate the t2no image and the regular image horizontally
            concatenated_img = np.hstack((img_t2no_multiplied, img_image))

            # Write the concatenated image as a frame to the video
            video_writer.write(concatenated_img)

            # Optionally display the concatenated images (uncomment to visualize)
            # cv2.imshow('Concatenated Video Simulation', concatenated_img)
            # if cv2.waitKey(50) & 0xFF == ord('q'):
            #     break  # Press 'q' to exit the video early

    # Release the video writer
    video_writer.release()

# Close any OpenCV windows (if opened)
cv2.destroyAllWindows()

print(f"Video saved at {video_output_path}")


ValueError: The number of images in the two folders does not match!